# NFNET Overview
![ImageNet](https://miro.medium.com/max/1400/1*CjpipU_oChc899f_Esjpyg.png)
 
NFNET's are Convolutional Residual Style Networks that have no batch normalization build in them. But without the batch normalization usually networks are not performing so well or cannot scale to larger batch sizes however NFNET builds networks that scale to large batch sizes and are more efficient than previous state-of-the-art methods. The training latency vs accuracy graph shows that NFnets are 8.7× times faster than EffNet-B7 for the same top-1 accuracy score trained on ImageNet. 

# What is Fastai?
> Fastai is a deep learning library which provides practitioners with high-level components that can quickly and easily provide state-of-the-art results in standard deep learning domains, and provides researchers with low-level components that can be mixed and matched to build new approaches. It aims to do both things without substantial compromises in ease of use, flexibility, or performance. This is possible thanks to a carefully layered architecture, which expresses common underlying patterns of many deep learning and data processing techniques in terms of decoupled abstractions. These abstractions can be expressed concisely and clearly by leveraging the dynamism of the underlying Python language and the flexibility of the PyTorch library.

# *Please upvote the kernel if you find it useful*

# Importing the necessary libraries:

In [ ]:
import sys; 
sys.path.insert(0,'../input/timm-nfnet')
import timm

In [ ]:
! pip install fastai==1.0.61

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

In [ ]:
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from pathlib import Path

from fastai.vision import *
from fastai.metrics import error_rate

from PIL import Image

In [ ]:
inputs=Path("../input/digit-recognizer")
os.listdir(inputs)

# Read the train and test datasets:

In [ ]:
# load training data and explore the first three rows
train=pd.read_csv(inputs/"train.csv")
train.head(3)

In [ ]:
# load test data and explore the first three rows
test=pd.read_csv(inputs/"test.csv")
test.head(3)

# Image transforms

**To get a set of transforms with default values that work pretty well in a wide range of tasks, it's often easiest to use get_transforms.**                                                                                                     
* **tfms is just a parameter used later during training, which is initalized here.** 
* **tr and te are paths to be used.**

In [ ]:
# tfms can be passed directly to define a DataBunch object (see below) which is then associated with a model to begin training.
tfms = get_transforms(do_flip=False) # if True the image is randomly flipped
tr=Path("../train")
te=Path("../test")

**We have to try and get the dataset into a folder format, from the existing format, which will make it easier to use fastai's functions.**

In [ ]:
for index in range(10):
    try:
        os.makedirs(tr/str(index))
    except:
        pass

In [ ]:
sorted(os.listdir(tr))

In [ ]:
try:
    os.makedirs(te)
except:
    pass


# Prepare Data
**Currently, it is not even an image, just a 0s and 1s, as seen from the training set. Using the functions below, we can convert them into images:**

In [ ]:
for index, row in train.iterrows():
    
    label,digit = row[0], row[1:]
    
    filepath = tr/str(label)
    filename = f"{index}.jpg"
    
    digit = digit.values
    digit = digit.reshape(28,28)
    digit = digit.astype(np.uint8)
    
    img = Image.fromarray(digit)
    img.save(filepath/filename)
    
    

In [ ]:
for index, digit in test.iterrows():

    filepath = te
    filename = f"{index}.jpg"
    
    digit = digit.values
    digit = digit.reshape(28,28)
    digit = digit.astype(np.uint8)
    
    img = Image.fromarray(digit)
    img.save(filepath/filename)

# Display images 

In [ ]:
def displayRandomImagesFromEveryFolder(directory=tr, samplesPerDigit=5):

    fig = plt.figure(figsize=(5,10))
    
    for rowIndex in range(1, 10):
        subdirectory = str(rowIndex)
        path = directory/subdirectory
        images = os.listdir(path)
        for sampleIndex in range(1,samplesPerDigit+1):
            randomNumber = random.randint(0, len(images)-1)
            image = Image.open(path/images[randomNumber])
            ax = fig.add_subplot(10, 5, samplesPerDigit*rowIndex + sampleIndex)
            ax.axis("off")
            
            plt.imshow(image, cmap='gray')
            
    
    plt.show()
    
displayRandomImagesFromEveryFolder()

**The dataset has been converted into images!**

**We can move on to getting the data from folders, and seperating them into training and validation sets. Also normalization is very important to make sure all values lie between 0 and 1.**

**It turns out that the PosixPath is not iterated by ImageDataBunch in Kaggle.So we can change the path created by pathlib library which was a PosixPath object to just a string which specifies the path to the training and testing directories, so in our case train path = "../train" and test path = "../test"**

In [ ]:
data = ImageDataBunch.from_folder(path="../train",test="../test",ds_tfms=tfms, valid_pct=0.2,bs=32,size=24).normalize(imagenet_stats)

In [ ]:
data.show_batch(rows=3 ,figsize=(5,5))

**The data has been successfully extracted from the folders.**

**We can also check what classes exist:**

In [ ]:
print(data.classes)

# Model 

NFNets are a family of modified ResNets that achieves competitive accuracies without batch normalization. To do so, it applies 3 different techniques:
* Modified residual branches and convolutions with Scaled Weight Standardization
* Adaptive Gradient Clipping
* Architecture optimization for improved accuracy and training spee

In [ ]:
class NFNetModel(nn.Module):
    
    def __init__(self, num_classes=10, model_name='nfnet_f1', pretrained=False):
        super(NFNetModel, self).__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        self.model.head.fc = nn.Linear(self.model.head.fc.in_features, num_classes)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
model = NFNetModel()

In [ ]:
learn = Learner(data, model, metrics=accuracy)

**Next, we can figure out what ideal learning rates are:**

In [ ]:
# find optimal learning rate and plot the graph
learn.lr_find()
# plot loss vs. learning rate
learn.recorder.plot()

**We can clearly see that the learning rate is most effective at 1e-03, but let's try without a predefined learning rate:**

In [ ]:
learn.fit_one_cycle(10)

**99% accuracy, not bad at all.**

**Saving this model:**

In [ ]:
learn.save("501")

**Now, let's try with the optimal learning rates:**

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10,max_lr=1e-3)

In [ ]:
learn.save("502")

# Results

**We can interpret our results as well:**

In [ ]:
interp=ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(8,8))

In [ ]:
interp.most_confused(min_val=3)

**From this, we can see which two numbers are confused most and the number of times.**

In [ ]:
interp.plot_top_losses(9,figsize=(7,7))

**These are the images which had the highest loss, that is the biggest difference between the probability of being corect and actually being correct.**

# Prediction

In [ ]:
class_score,y=learn.get_preds(DatasetType.Test)

In [ ]:
probs= class_score[0].tolist()
[f"{index}: {probs[index]}" for index in range(len(probs))]

**These are the probabilities that the image is any of these numbers. But we don't want that. We only want the highest probability:**

In [ ]:
class_score=np.argmax(class_score,axis=1)

In [ ]:
class_score[0].item()

# Submission

Now, creating the submission file based on the example given (which should contain ImageId and Label):

In [ ]:
samplesub=pd.read_csv(inputs/"sample_submission.csv")
samplesub.head()

In [ ]:
ImageId = [os.path.splitext(path)[0] for path in os.listdir(te)]
ImageId = [int(path) for path in ImageId]
ImageId = [ID+1 for ID in ImageId]
ImageId[:5]

In [ ]:
subs=pd.DataFrame({"ImageId":ImageId,"Label":class_score})

In [ ]:
subs.to_csv("submission.csv",index=False)
subs.head(3)